In [1]:
from pyspark import SparkConf, SparkContext
# SparkConf : 환경 설정
# SparkContext : 실제 개발 프로그램. 드라이브
#conf = SparkConf().setMaster("yarn") # yarn으로 하면 자동으로 하둡을 사용하게 된다
conf = SparkConf().setMaster("local").setAppName("key-value_rdd_op_joins")
sc = SparkContext(conf=conf) # sc 변수명 바꾸면 안된다. 실무에서는 스파크 서브밋 이라는 것을 사용한다. 그때 필수

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/25 19:57:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Operations
- `groupByKey`
    - `KeyValueRDD.groupByKey()`
    - 그룹핑 후에 특정 Transformations 연산
    - `Key`값이 존재하는 상태로 시작

In [3]:
rdd = sc.parallelize([
    ("짜장면", 15),
    ("짬뽕",  10),
    ("짜장면", 5)
])

In [4]:
g_rdd = rdd.groupByKey()
g_rdd

PythonRDD[5] at RDD at PythonRDD.scala:53

In [6]:
g_rdd.collect() # 키로 묶여지는 것까지됨. 이후에 특정 transformation  연산을 연결함

[('짜장면', <pyspark.resultiterable.ResultIterable at 0x7f6780356610>),
 ('짬뽕', <pyspark.resultiterable.ResultIterable at 0x7f6780356910>)]

In [7]:
g_rdd.mapValues(len).collect()

[('짜장면', 2), ('짬뽕', 1)]

In [10]:
g_rdd.mapValues(list).collect()

[('짜장면', [15, 5]), ('짬뽕', [10])]

# Joins

In [12]:
rdd1 = sc.parallelize([
    ("foo", 1),
    ("goo", 2),
    ("hoo", 3)
])

rdd2 = sc.parallelize([
    ("foo", 1),
    ("goo", 2),
    ("goo", 10),
    ("moo", 6),
])

In [13]:
# Inner Join : 서로 간에 존재하는 키만 합쳐준다.
rdd1.join(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10))]

In [14]:
# Outer Join
# 기준이 되는 한 쪽에는 데이터가 존재하고, 다른 쪽에는 데이터가 없는 경우 기준 쪽의 데이터가 항상 남아있다.

# leftOuterJoin : 왼쪽에 있는 RDD가 기준이 된다. (함수를 호출하는 쪽)
rdd1.leftOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10)), ('hoo', (3, None))]

In [15]:
# rightOuterJoin : 오른쪽에 있는 RDD가 기준이 된다. 함수의 매개변수로 들어가는 쪽
rdd1.rightOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('moo', (None, 6)), ('goo', (2, 2)), ('goo', (2, 10))]

In [16]:
sc.stop()